In [1]:
# !pip install ollama

In [1]:
import ollama
import pandas as pd


In [2]:
df = pd.read_csv('arabot_1k_translated_random.csv')
df.head()

,Description_English,Type
0,3100 riyals were transferred to a quorum accou...,Payment Operations
1,A bank transfer was transferred to a friend on...,Payment Operations
2,A broken incoming transfer. The amount of 568 ...,Payment Operations
3,A company's amount was withdrawn twice on a da...,Bank Cards
4,A complaint has already been revealed and I ha...,Financing & Leasing


In [3]:
types = df['Type'].unique().tolist()

In [4]:
def zero_shot_classification(description, types):
    prompt = f"""Given the following text: "{description}"
    
    Classify it into one of the following categories:
    {', '.join(types)}
    
    Output only the category name, nothing else."""
    
    response = ollama.chat(model='llama3:70b', messages=[{'role': 'user', 'content': prompt}])
    return response['message']['content'].strip()

In [7]:
zero_shot_classification(df['Description_English'][0], types)

'Payment Operations'

6 minutes is a long inference time for one row. Based on this, if we were to do it on 1,000 rows, it would take us 4+ days to run everthing. Hence I will calculate the zero-shot accuracy based on 30 rows of data, which should take around 3-4 hours to run. 

In [11]:
df['Predicted_Type'] = df['Description_English'][0:30].apply(lambda x: zero_shot_classification(x, types))

In [12]:
df.head()

,Description_English,Type,Predicted_Type
0,3100 riyals were transferred to a quorum accou...,Payment Operations,Payment Operations
1,A bank transfer was transferred to a friend on...,Payment Operations,Payment Operations
2,A broken incoming transfer. The amount of 568 ...,Payment Operations,Payment Operations
3,A company's amount was withdrawn twice on a da...,Bank Cards,Payment Operations
4,A complaint has already been revealed and I ha...,Financing & Leasing,Payment Operations


In [15]:
accuracy = (df['Type'][0:30] == df['Predicted_Type'][0:30]).mean()
print(f"Zero-Shot llama3 Classification Accuracy: {accuracy * 100:.1f}%")

Zero-Shot llama3 Classification Accuracy: 50.0%
